# 2024 CITS4012 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

## imports

In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger_eng")

stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


## Dataset Path




In [2]:
db_path = "dataset"

train_file = f"{db_path}/train.json"
test_file = f"{db_path}/test.json"
val_file = f"{db_path}/train.json"

# 1.Dataset Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [3]:
import json
with open(train_file, "r") as fp:
    train = json.load(fp)
with open(val_file, "r") as fp:
    val = json.load(fp)
with open(test_file, "r") as fp:
    test = json.load(fp)

train['premise']['0'], train['hypothesis']['0'], train['label']['0'], len(train['premise'])



('Pluto rotates once on its axis every 6.39 Earth days;',
 'Earth rotates on its axis once times in one day.',
 'neutral',
 23088)

In [4]:
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk import pos_tag
import numpy as np


class Config:
    stem = False
    stemmer = PorterStemmer()
    lemmatize = False
    lemmatizer = WordNetLemmatizer()
    tokenize = True

    clean_text = True
    regex = r"[^a-z0-9\-\s]"

    pos = False
    pos_tagger = pos_tag

    lower = True

    stopwords = set(stopwords.words('english'))


class textProcesser():
    def __init__(self, cfg: Config) -> None:
        self.cfg = cfg

        self.lemmatizer = cfg.lemmatizer
        self.stemmer = cfg.stemmer
        self.stop_words = cfg.stopwords


    def clean(self, sentence: str) -> list[str]:
        tokens = [sentence]
        if self.cfg.tokenize:
            tokens = nltk.word_tokenize(sentence)

        if self.cfg.lemmatize:
            tokens = [self.lemmatizer.lemmatize(token) for token in tokens]

        if self.cfg.stem:
            tokens = [self.stemmer.stem(token) for token in tokens]

        if self.cfg.clean_text:
                
            tokens = [re.sub(self.cfg.regex, '', token.lower() if self.cfg.lower else token) for token in tokens]
            tokens = [token for token in tokens if token and token not in self.stop_words]

        if self.cfg.pos:
            tokens = self.cfg.pos_tagger(tokens)
        
        return tokens
    
    def clean_text(self, json):
        clean = {'premise': {}, 'hypothesis': {}, 'label': {}}
        for idx, key in enumerate(json['label'].keys()):
            if 'neutral' in json['label'][key]:
                clean['label'][idx] = 0
            else:
                clean['label'][idx] = 1

            clean['premise'][idx] = self.clean(json['premise'][key])
            clean['hypothesis'][idx] = self.clean(json['hypothesis'][key])
            
        return clean

In [5]:
clean_train = textProcesser(Config()).clean_text(train)
clean_val = textProcesser(Config()).clean_text(val)
clean_test = textProcesser(Config()).clean_text(test)
clean_train['premise'][2], clean_train['hypothesis'][2], clean_train['label'][2]

(['geysers', '-', 'periodic', 'gush', 'hot', 'water', 'surface', 'earth'],
 ['surface', 'sun', 'much', 'hotter', 'almost', 'anything', 'earth'],
 0)

In [6]:
import pandas as pd

train_df = pd.DataFrame({
    "label": clean_train["label"],
    "premise": clean_train["premise"],
    "hypothesis": clean_train["hypothesis"]
})

# reset index if needed
train_df = train_df.reset_index(drop=True)
train_df.head()

,label,premise,hypothesis
0,0,"[pluto, rotates, axis, every, 639, earth, days]","[earth, rotates, axis, times, one, day]"
1,1,"[--, -glenn, per, day, earth, rotates, axis]","[earth, rotates, axis, times, one, day]"
2,0,"[geysers, -, periodic, gush, hot, water, surfa...","[surface, sun, much, hotter, almost, anything,..."
3,1,"[facts, liquid, water, droplets, changed, invi...","[evaporation, responsible, changing, liquid, w..."
4,1,"[comparison, earth, rotates, axis, per, day, r...","[earth, rotates, axis, times, one, day]"


In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,  DataLoader

class nliDataset(Dataset):
  def __init__(self, df, embed_model):
    self.df = df
    self.embed_model = embed_model

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    premise = self.df["premise"].iloc[index]
    hypothesis = self.df["hypothesis"].iloc[index]
    p_embed = np.array([self.embed_model.wv[word] for word in premise if word in self.embed_model.wv])
    h_embed = np.array([self.embed_model.wv[word] for word in hypothesis if word in self.embed_model.wv])
    label = int(self.df["label"].iloc[index])

    p_embed = torch.tensor(p_embed, dtype=torch.float32)
    h_embed = torch.tensor(h_embed, dtype=torch.float32)
    label = torch.tensor(label, dtype=torch.long)

    return p_embed, h_embed, label


In [8]:
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    p_embeds, h_embeds, labels = zip(*batch)

    # Compute lengths for each sequence
    p_lengths = torch.tensor([x.shape[0] for x in p_embeds], dtype=torch.long)
    h_lengths = torch.tensor([x.shape[0] for x in h_embeds], dtype=torch.long)

    p_padded = pad_sequence(p_embeds, batch_first=True)  # (batch, max_p_len, embed_dim)
    h_padded = pad_sequence(h_embeds, batch_first=True)  # (batch, max_h_len, embed_dim)

    labels = torch.stack(labels)

    return p_padded, h_padded, labels, p_lengths, h_lengths


In [9]:
from gensim.models import Word2Vec
sentences = list(clean_train['premise'].values()) + list(clean_train['hypothesis'].values())

embed_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=3, workers=2, sg=1)


In [10]:
train_db = nliDataset(train_df, embed_model=embed_model)
loader = DataLoader(train_db, batch_size=8, shuffle=True, collate_fn=collate_fn)


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## BiLSTM

In [25]:
from torch.nn.utils.rnn import pack_padded_sequence

class twin_LSTM(nn.Module):
  def __init__(self, input_size=100, hidden_size=128, output_size=2, dropout=0.5, bidirectional=True):
    super().__init__()
    self.hidden_size = hidden_size

    self.px_lstm = nn.LSTM(input_size, self.hidden_size, num_layers=1, bidirectional=False, batch_first=True)
    self.hx_lstm = nn.LSTM(input_size, self.hidden_size, num_layers=1, bidirectional=False, batch_first=True)

    self.fc = nn.Sequential(
      nn.Dropout(dropout),
      nn.Linear(self.hidden_size * (2 if bidirectional else 1), output_size)
    )

  def forward(self, px, hx, plengths, hlengths):

    packed_px = pack_padded_sequence(px, plengths.cpu(), batch_first=True, enforce_sorted=False)
    px_output, (px_hn, _) = self.px_lstm(packed_px)

    packed_hx = pack_padded_sequence(hx, hlengths.cpu(), batch_first=True, enforce_sorted=False)
    hx_output, (hx_hn, _) = self.hx_lstm(packed_hx)

    px_out = px_hn[-1]
    hx_out = hx_hn[-1]


    out = self.fc(torch.cat([px_out, hx_out], dim=1))
    return out


In [26]:
twin = twin_LSTM(input_size=100, hidden_size=128, output_size=2, dropout=0.5, bidirectional=True)

for batch in loader:
    p_padded, h_padded, labels, p_lengths, h_lengths = batch
    outputs = twin(p_padded, h_padded, p_lengths, h_lengths)
    print(outputs.shape)
    break

torch.Size([8, 2])


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*